In [12]:
import os
import sys
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import cv2 as cv

from keras.layers import Flatten, Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.losses import categorical_crossentropy
from keras.utils import img_to_array
from keras.utils.image_utils import smart_resize

import tensorflow as tf

print(f'Python: {sys.version}')

Python: 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


In [13]:
RANDOM_STATE = 0

In [14]:
# Working with pre trained model 

#base_model = MobileNet(input_shape=(224, 224, 3), include_top=False)
base_model = MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)


for layer in base_model.layers:
  layer.trainable = False


x = Flatten()(base_model.output)
x = Dense(units=2 , activation='softmax')(x)

# creating our model.
model = Model(base_model.input, x)

In [15]:
model.compile(optimizer='adam', loss=categorical_crossentropy , metrics=['accuracy'])

In [16]:
sessions = []
directory_template = './data/images/images/train/$session$/'
for session in os.listdir('./data/images/images/train'):
    # Find XML filename
    directory = directory_template.replace('$session$', session)
    files = os.listdir(directory)
    
    for file in files:
        # Load JPG filename
        jpg_filename = directory + file
        
        emotion_dict = {
            'emotion': session,
            'img': jpg_filename
        }
        sessions.append(emotion_dict)


In [17]:
df_sessions = pd.DataFrame(sessions)
df_sessions.index.rename('session', inplace=True)
df_sessions["aus"] = 0
df_sessions.info

<bound method DataFrame.info of           emotion                                           img  aus
session                                                             
0           angry        ./data/images/images/train/angry/0.jpg    0
1           angry        ./data/images/images/train/angry/1.jpg    0
2           angry       ./data/images/images/train/angry/10.jpg    0
3           angry    ./data/images/images/train/angry/10002.jpg    0
4           angry    ./data/images/images/train/angry/10016.jpg    0
...           ...                                           ...  ...
28816    surprise  ./data/images/images/train/surprise/9969.jpg    0
28817    surprise  ./data/images/images/train/surprise/9985.jpg    0
28818    surprise  ./data/images/images/train/surprise/9990.jpg    0
28819    surprise  ./data/images/images/train/surprise/9992.jpg    0
28820    surprise  ./data/images/images/train/surprise/9996.jpg    0

[28821 rows x 3 columns]>

In [18]:

df_aus = pd.read_excel('data/aus.xlsx')

In [19]:
df_anger = df_sessions.loc[lambda df: df['emotion'] == "angry"].reset_index()
df_disgust = df_sessions.loc[lambda df: df['emotion'] == "disgust"].reset_index()
df_happy = df_sessions.loc[lambda df: df['emotion'] == "happy"].reset_index()
df_fear = df_sessions.loc[lambda df: df['emotion'] == "fear"].reset_index()
df_surprise = df_sessions.loc[lambda df: df['emotion'] == "surprise"].reset_index()
df_sadness = df_sessions.loc[lambda df: df['emotion'] == "sad"].reset_index()
df_neutral = df_sessions.loc[lambda df: df['emotion'] == "neutral"].reset_index()

In [20]:
import random
list_anger = ["4,5,7,17,23", "4,5,7,17,24"]
list_disgust = ["9,17", "10, 17"]
list_sadness = ["1,4,11,15,54,64","1,4,15","6,15,54,64","11,17","1,4,15,17"]
list_fear = ["1,2,4,5","1,2,4,5,25"]
list_surprise=["1,2,5,26", "1,2,26"]
list_happy = ["6,12"]
list_neutral = ["0"]

#for i in range(len(df_anger.values)):
#      rand_idx = random.randrange(len(list_anger))
#      random_num = list_anger[rand_idx]

#      df_anger["aus"][i] = random_num

# i = 0
for i in range(len(df_disgust.values)):
     rand_idx = random.randrange(len(list_disgust))
     random_num = list_disgust[rand_idx]

     df_disgust["aus"][i] = random_num

# i = 0
# for i in range(len(df_sadness.values)):
#     rand_idx = random.randrange(len(list_sadness))
#     random_num = list_sadness[rand_idx]

#     df_sadness["aus"][i] = random_num

# i = 0
# for i in range(len(df_fear.values)):
#     rand_idx = random.randrange(len(list_fear))
#     random_num = list_fear[rand_idx]

#     df_fear["aus"][i] = random_num

# i = 0
# for i in range(len(df_surprise.values)):
#      rand_idx = random.randrange(len(list_surprise))
#      random_num = list_surprise[rand_idx]

#      df_surprise["aus"][i] = random_num


# df_anger["aus"] = list_anger[2]
# df_disgust["aus"] = list_disgust[0]
# df_sadness["aus"] = list_sadness[0]
# df_fear["aus"] = list_fear[0]
# df_surprise["aus"] = list_surprise[0]
# df_happy["aus"] = list_happy[0]
# df_neutral["aus" ] = list_neutral[0]

C:\Users\rodol\AppData\Local\Temp\ipykernel_9968\3691740048.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_disgust["aus"][i] = random_num
C:\Users\rodol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [21]:
#df_modelo = pd.concat([df_anger])
df_disgust.to_csv("base_aus_disgust.csv")

In [22]:
df_modelo = pd.read_csv("base_aus_disgust.csv")
df_modelo = df_modelo.drop(columns=["Unnamed: 0", "session"])

In [23]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_modelo, test_size=0.3)
train


,emotion,img,aus
59,disgust,./data/images/images/train/disgust/14652.jpg,"9,17"
141,disgust,./data/images/images/train/disgust/19911.jpg,"9,17"
49,disgust,./data/images/images/train/disgust/13551.jpg,"9,17"
391,disgust,./data/images/images/train/disgust/7313.jpg,"10, 17"
76,disgust,./data/images/images/train/disgust/15780.jpg,"10, 17"
...,...,...,...
405,disgust,./data/images/images/train/disgust/8379.jpg,"10, 17"
420,disgust,./data/images/images/train/disgust/9159.jpg,"9,17"
238,disgust,./data/images/images/train/disgust/27771.jpg,"10, 17"
26,disgust,./data/images/images/train/disgust/11971.jpg,"10, 17"


In [24]:
train_datagen = ImageDataGenerator(
    zoom_range = 0.2, 
    shear_range = 0.2, 
    horizontal_flip=True, 
    rescale = 1./255
)

train_data = train_datagen.flow_from_dataframe(
    dataframe = pd.concat([train]).sample(frac=1),
    directory=".",
    x_col = "img",
    y_col = "aus",
    subset = "training",
    batch_size = 32,
    seed = 42,
    class_mode = "categorical",
    target_size = (48,48)
)

test_data = train_datagen.flow_from_dataframe(
    dataframe = pd.concat([test]).sample(frac=1),
    directory=".",
    x_col = "img",
    y_col = "aus",
    subset = "training",
    batch_size = 32,
    seed = 42,
    class_mode = "categorical",
    target_size = (48,48)
)

Found 305 validated image filenames belonging to 2 classes.
Found 131 validated image filenames belonging to 2 classes.


In [25]:
hist = model.fit(
    train_data,
    steps_per_epoch=10,
    epochs=30,
    validation_data=test_data,
    validation_steps=8,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            filepath='./results/model_aus_disgust.h5',
            save_best_only=True,
            monitor="accuracy"
        )
    ]
)

print()
max_acc = max(hist.history['accuracy']) * 100
print(f'Accuracy: {max_acc = :.2f}%')

Epoch 1/30
10/10 [==============================] - 5s 348ms/step - loss: 0.6933 - accuracy: 0.5082 - val_loss: 0.6927 - val_accuracy: 0.5420
Epoch 2/30
10/10 [==============================] - 1s 50ms/step - loss: 0.6931 - accuracy: 0.5082
Epoch 3/30
10/10 [==============================] - 1s 52ms/step - loss: 0.6931 - accuracy: 0.5082
Epoch 4/30
10/10 [==============================] - 1s 63ms/step - loss: 0.6932 - accuracy: 0.5082
Epoch 5/30
10/10 [==============================] - 1s 56ms/step - loss: 0.6933 - accuracy: 0.5082
Epoch 6/30
10/10 [==============================] - 1s 56ms/step - loss: 0.6930 - accuracy: 0.5082
Epoch 7/30
10/10 [==============================] - 0s 47ms/step - loss: 0.6930 - accuracy: 0.5082
Epoch 8/30
10/10 [==============================] - 1s 61ms/step - loss: 0.6930 - accuracy: 0.5082
Epoch 9/30
10/10 [==============================] - 1s 53ms/step - loss: 0.6930 - accuracy: 0.5082
Epoch 10/30
10/10 [==============================] - 0s 46ms/step 

In [26]:
labels = np.sort(df_modelo['aus'].unique())
img_cv = cv.imread('./data/images/images/validation/disgust/533.jpg', cv.IMREAD_COLOR)
img_keras = img_to_array(img_cv)
img_keras = np.expand_dims(img_keras, axis=0)
img_keras = smart_resize(img_keras, (224, 224))
prediction = model.predict(img_keras)[0]
mean = prediction.mean()
std = prediction.std()
prediction_idx = np.argmax(prediction)
prediction_label = labels[prediction_idx]
prediction_prob = prediction[prediction_idx]

print(f'{prediction_label}: {round(prediction_prob * 100, 2)}%')

1/1 [==============================] - 1s 641ms/step
10, 17: 51.3%
